In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import levene, bartlett
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
data_anxity = pd.read_csv("anxity.csv")
data_anxity

In [16]:
anxity_subset = data_anxity[["group","observation","reads"]]
anxity_subset

,group,observation,reads
0,grp1,t1,14.1
1,grp1,t2,14.4
2,grp1,t3,14.1
3,grp1,t1,14.5
4,grp1,t2,14.6
...,...,...,...
130,grp3,t2,16.4
131,grp3,t3,15.1
132,grp3,t1,19.0
133,grp3,t2,17.2


In [33]:
## Normality check
statistic, p_value = stats.shapiro(data_anxity["reads"])
print("statistic:", statistic)
print("p_value:", p_value)

statistic: 0.9869789036748995
p_value: 0.23155222959112665


In [42]:
# Extract unique levels of "observation"
observation_levels = data_anxity["observation"].unique()
observation_levels

array(['t1', 't2', 't3'], dtype=object)

In [51]:
from scipy.stats import levene

# Group the "reads" data by "observation"
grouped_reads = [data_anxity[data_anxity["observation"] == level]["reads"] for level in ["t1", "t2", "t3"]]

# Perform Levene's test
statistic, p_value = levene(*grouped_reads)

print("Levene's test statistic:", statistic)
print("p-value:", p_value)


Levene's test statistic: 2.3410322270615245
p-value: 0.10021171478874775


#### there is no significant mean difference between the groups
#### Ha = there is significant difference between the groups
#### alpha = 0.05
Model we are using is OLS from statsmodels package.
In ANOVA, OLS is used to fit the model to the observed data, and it estimates the mean response for each level of the categorical variables (factors) by finding the best-fitting line through the data points.

In [19]:
mod = ols("reads ~ observation", data= anxity_subset).fit()

In [20]:
aov_table = sm.stats.anova_lm(mod, type=2)
print(aov_table)

                df      sum_sq    mean_sq          F    PR(>F)
observation    2.0   66.578815  33.289407  10.839924  0.000044
Residual     132.0  405.372000   3.071000        NaN       NaN


results shows us that p_value is significant between the groups. but we don't know which group have the significance difference. to know about the significant group. we do the TukeyHSD test.

In [24]:
#pairwise_tukeyhsd test
tukey_result = sm.stats.multicomp.pairwise_tukeyhsd(data_anxity['reads'], data_anxity['observation'])
print(tukey_result)


Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    t1     t2    -0.78 0.0914 -1.6557 0.0957  False
    t1     t3  -1.7178    0.0 -2.5935 -0.842   True
    t2     t3  -0.9378 0.0327 -1.8135 -0.062   True
---------------------------------------------------


In [27]:
import pingouin as pg
aov = pg.anova(data_anxity, dv = "reads", between = "observation", detailed= True)
print(aov_table)

                df      sum_sq    mean_sq          F    PR(>F)
observation    2.0   66.578815  33.289407  10.839924  0.000044
Residual     132.0  405.372000   3.071000        NaN       NaN


In [28]:
pt = pg.pairwise_tukey(dv = "reads", between= "observation", data= data_anxity)
print(pt)

    A   B    mean(A)    mean(B)      diff        se         T   p-tukey  \
0  t1  t2  16.915556  16.135556  0.780000  0.369444  2.111281  0.091421   
1  t1  t3  16.915556  15.197778  1.717778  0.369444  4.649629  0.000024   
2  t2  t3  16.135556  15.197778  0.937778  0.369444  2.538349  0.032734   

    hedges  
0  0.47325  
1  0.97601  
2  0.49713  
